In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [3]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [4]:
ctable = CharacterTable(chars)

In [5]:
print (ctable.chars)

[' ', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [6]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    a, b = max(a,b), min(a,b)
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print (questions[:5],expected[:5])

Generating data...
['73-4   ', '840-174', '84-9   ', '43-8   ', '7-1    '] ['69  ', '666 ', '75  ', '35  ', '6   ']


In [7]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

print (x[:5],y[:5])

Vectorization...
[[[False False False False False False False False False  True False
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False False False  True False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False 

In [8]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10

(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

In [9]:
print('Build model...' )
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [10]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        #print(rowx)
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 235us/step - loss: 1.8882 - acc: 0.3250 - val_loss: 1.7435 - val_acc: 0.3498
Q 676-158 T 518  ☒ 123 
Q 418-372 T 46   ☒ 15  
Q 103-42  T 61   ☒ 15  
Q 504-91  T 413  ☒ 153 
Q 851-68  T 783  ☒ 153 
Q 460-80  T 380  ☒ 153 
Q 385-230 T 155  ☒ 153 
Q 586-76  T 510  ☒ 153 
Q 746-4   T 742  ☒ 15  
Q 846-72  T 774  ☒ 153 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 153us/step - loss: 1.7175 - acc: 0.3542 - val_loss: 1.7017 - val_acc: 0.3596
Q 494-26  T 468  ☒ 170 
Q 220-58  T 162  ☒ 10  
Q 33-19   T 14   ☒ 10  
Q 810-33  T 777  ☒ 100 
Q 934-303 T 631  ☒ 170 
Q 529-81  T 44

18000/18000 [==============================] - 3s 147us/step - loss: 1.2164 - acc: 0.5521 - val_loss: 1.2226 - val_acc: 0.5455
Q 649-20  T 629  ☒ 645 
Q 564-12  T 552  ☒ 555 
Q 770-589 T 181  ☒ 105 
Q 750-37  T 713  ☒ 744 
Q 964-4   T 960  ☒ 953 
Q 945-886 T 59   ☒ 12  
Q 304-74  T 230  ☒ 264 
Q 280-70  T 210  ☒ 234 
Q 514-16  T 498  ☒ 585 
Q 582-32  T 550  ☒ 542 

--------------------------------------------------
Iteration 14
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 145us/step - loss: 1.1905 - acc: 0.5614 - val_loss: 1.1946 - val_acc: 0.5550
Q 49-12   T 37   ☒ 39  
Q 796-617 T 179  ☒ 129 
Q 614-49  T 565  ☒ 566 
Q 476-373 T 103  ☒ 11  
Q 850-342 T 508  ☒ 529 
Q 831-9   T 822  ☒ 832 
Q 274-23  T 251  ☒ 249 
Q 904-9   T 895  ☒ 939 
Q 529-81  T 448  ☒ 467 
Q 740-58  T 682  ☒ 646 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 141us/step - loss: 0.8483 - acc: 0.6877 - val_loss: 0.8937 - val_acc: 0.6600
Q 972-843 T 129  ☒ 123 
Q 133-61  T 72   ☒ 82  
Q 565-424 T 141  ☒ 131 
Q 771-30  T 741  ☒ 749 
Q 683-90  T 593  ☒ 696 
Q 440-433 T 7    ☒ 6   
Q 416-55  T 361  ☒ 360 
Q 983-0   T 983  ☒ 984 
Q 523-2   T 521  ☒ 529 
Q 750-159 T 591  ☑ 591 

--------------------------------------------------
Iteration 28
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 160us/step - loss: 0.8297 - acc: 0.6958 - val_loss: 0.8666 - val_acc: 0.6775
Q 988-73  T 915  ☒ 923 
Q 651-61  T 590  ☒ 593 
Q 727-645 T 82   ☒ 11  
Q 165-28  T 137  ☒ 135 
Q 460-91  T 369  ☒ 377 
Q 798-138 T 660  ☒ 675 
Q 185-9   T 176  ☒ 171 
Q 442-70  T 372  ☒ 376 
Q 856-40  T 816  ☒ 824 
Q 613-432 T 181  ☒ 187 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 140us/step - loss: 0.4329 - acc: 0.8549 - val_loss: 0.4616 - val_acc: 0.8341
Q 857-105 T 752  ☒ 753 
Q 79-41   T 38   ☑ 38  
Q 141-10  T 131  ☒ 132 
Q 855-72  T 783  ☑ 783 
Q 636-5   T 631  ☑ 631 
Q 41-32   T 9    ☒ 10  
Q 190-72  T 118  ☒ 117 
Q 542-462 T 80   ☒ 76  
Q 451-220 T 231  ☒ 221 
Q 359-211 T 148  ☒ 138 

--------------------------------------------------
Iteration 42
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 139us/step - loss: 0.3970 - acc: 0.8714 - val_loss: 0.4333 - val_acc: 0.8451
Q 593-381 T 212  ☑ 212 
Q 692-2   T 690  ☒ 691 
Q 737-9   T 728  ☑ 728 
Q 119-83  T 36   ☒ 44  
Q 654-36  T 618  ☑ 618 
Q 431-107 T 324  ☒ 325 
Q 968-152 T 816  ☒ 826 
Q 881-749 T 132  ☒ 123 
Q 635-31  T 604  ☒ 614 
Q 660-8   T 652  ☑ 652 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 170us/step - loss: 0.1423 - acc: 0.9650 - val_loss: 0.1910 - val_acc: 0.9376
Q 644-44  T 600  ☒ 590 
Q 923-50  T 873  ☑ 873 
Q 471-92  T 379  ☒ 389 
Q 483-79  T 404  ☑ 404 
Q 292-226 T 66   ☑ 66  
Q 778-33  T 745  ☑ 745 
Q 842-77  T 765  ☑ 765 
Q 806-64  T 742  ☑ 742 
Q 974-57  T 917  ☑ 917 
Q 305-96  T 209  ☒ 219 

--------------------------------------------------
Iteration 56
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 160us/step - loss: 0.1413 - acc: 0.9633 - val_loss: 0.1769 - val_acc: 0.9421
Q 41-27   T 14   ☑ 14  
Q 669-29  T 640  ☑ 640 
Q 351-8   T 343  ☑ 343 
Q 219-93  T 126  ☑ 126 
Q 983-37  T 946  ☑ 946 
Q 884-556 T 328  ☒ 338 
Q 643-81  T 562  ☑ 562 
Q 511-93  T 418  ☑ 418 
Q 333-22  T 311  ☑ 311 
Q 99-67   T 32   ☑ 32  

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 153us/step - loss: 0.0814 - acc: 0.9777 - val_loss: 0.1265 - val_acc: 0.9565
Q 149-91  T 58   ☒ 68  
Q 881-666 T 215  ☑ 215 
Q 501-177 T 324  ☑ 324 
Q 800-0   T 800  ☑ 800 
Q 677-40  T 637  ☑ 637 
Q 432-23  T 409  ☑ 409 
Q 515-0   T 515  ☑ 515 
Q 960-40  T 920  ☑ 920 
Q 882-78  T 804  ☑ 804 
Q 670-475 T 195  ☑ 195 

--------------------------------------------------
Iteration 70
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 153us/step - loss: 0.0786 - acc: 0.9793 - val_loss: 0.1177 - val_acc: 0.9601
Q 673-460 T 213  ☑ 213 
Q 843-818 T 25   ☒ 24  
Q 480-406 T 74   ☑ 74  
Q 486-407 T 79   ☒ 89  
Q 172-27  T 145  ☑ 145 
Q 519-15  T 504  ☒ 404 
Q 708-622 T 86   ☑ 86  
Q 732-3   T 729  ☑ 729 
Q 342-86  T 256  ☑ 256 
Q 46-45   T 1    ☒ 0   

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 141us/step - loss: 0.0431 - acc: 0.9900 - val_loss: 0.1306 - val_acc: 0.9519
Q 543-65  T 478  ☑ 478 
Q 793-2   T 791  ☑ 791 
Q 601-29  T 572  ☒ 582 
Q 72-30   T 42   ☑ 42  
Q 830-50  T 780  ☑ 780 
Q 623-97  T 526  ☑ 526 
Q 142-19  T 123  ☒ 124 
Q 626-255 T 371  ☑ 371 
Q 243-168 T 75   ☒ 74  
Q 319-49  T 270  ☑ 270 

--------------------------------------------------
Iteration 84
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 156us/step - loss: 0.0793 - acc: 0.9753 - val_loss: 0.1230 - val_acc: 0.9576
Q 805-651 T 154  ☑ 154 
Q 470-55  T 415  ☑ 415 
Q 751-312 T 439  ☑ 439 
Q 981-547 T 434  ☑ 434 
Q 626-255 T 371  ☑ 371 
Q 708-254 T 454  ☑ 454 
Q 956-792 T 164  ☒ 154 
Q 996-65  T 931  ☑ 931 
Q 342-309 T 33   ☑ 33  
Q 460-108 T 352  ☑ 352 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 157us/step - loss: 0.0231 - acc: 0.9963 - val_loss: 0.0654 - val_acc: 0.9776
Q 662-254 T 408  ☑ 408 
Q 313-31  T 282  ☑ 282 
Q 413-37  T 376  ☑ 376 
Q 626-195 T 431  ☑ 431 
Q 428-74  T 354  ☑ 354 
Q 804-8   T 796  ☑ 796 
Q 860-196 T 664  ☑ 664 
Q 912-349 T 563  ☑ 563 
Q 737-393 T 344  ☑ 344 
Q 574-0   T 574  ☑ 574 

--------------------------------------------------
Iteration 98
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 142us/step - loss: 0.0212 - acc: 0.9969 - val_loss: 0.0661 - val_acc: 0.9766
Q 80-51   T 29   ☑ 29  
Q 635-31  T 604  ☑ 604 
Q 306-44  T 262  ☑ 262 
Q 898-4   T 894  ☑ 894 
Q 486-8   T 478  ☑ 478 
Q 996-65  T 931  ☑ 931 
Q 669-520 T 149  ☒ 159 
Q 436-6   T 430  ☑ 430 
Q 984-22  T 962  ☑ 962 
Q 916-795 T 121  ☒ 110 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

In [11]:
print("MSG : Prediction")
test_x = ["555-275", "860-7  ", "340-29 "]
test_y = ["280 ", "853 ", "311 "]
x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
for j, (i, c) in enumerate(zip(test_x, test_y)):
    x[j] = ctable.encode(i, MAXLEN)
    y[j] = ctable.encode(c, DIGITS + 1)    
x.shape

MSG : Prediction


(3, 7, 12)

In [12]:
right = 0
preds = model.predict_classes(x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(x[i])
    correct = ctable.decode(y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 555-275 T 280  ☑ 280 
Q 860-7   T 853  ☑ 853 
Q 340-29  T 311  ☑ 311 
MSG : Accuracy is 1.0
